<a href="https://colab.research.google.com/github/DhruvAjayToshniwal/Genetic-Algorithm-for-Financial-Portfolio-Optimization/blob/main/Genetic%20Algorithm%20for%20Financial%20Portfolio%20Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install yfinance deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 8.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import yfinance as yf
import random
from deap import creator, base, tools, algorithms
import numpy as np

# List of fintech stocks
fintech_stocks = ['V', 'MA', 'PYPL', 'SQ', 'ADP', 'GS', 'MS', 'JPM']

# Fetch data function
def fetch_data(symbol, start, end):
    data = yf.download(symbol, start=start, end=end)
    return data

In [4]:
# Moving Average Crossover strategy function
def moving_average_crossover(data, short_window, long_window):
    signals = pd.DataFrame(index=data.index)
    signals['signal'] = 0.0
    signals['short_mavg'] = data['Close'].rolling(window=short_window, min_periods=1, center=False).mean()
    signals['long_mavg'] = data['Close'].rolling(window=long_window, min_periods=1, center=False).mean()
    signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:]
                                                > signals['long_mavg'][short_window:], 1.0, 0.0)
    signals['positions'] = signals['signal'].diff()
    return signals

In [5]:
# Returns calculation function
def calculate_returns(data, signals):
    positions = pd.DataFrame(index=signals.index).fillna(0.0)
    positions[data.columns[0]] = 100*signals['signal']   # This buys 100 shares
    portfolio = positions.multiply(data['Adj Close'], axis=0)
    pos_diff = positions.diff()
    portfolio['holdings'] = (positions.multiply(data['Adj Close'], axis=0)).sum(axis=1)
    portfolio['cash'] = 10000 - (pos_diff.multiply(data['Adj Close'], axis=0)).sum(axis=1).cumsum()
    portfolio['total'] = portfolio['cash'] + portfolio['holdings']
    portfolio['returns'] = portfolio['total'].pct_change()
    return portfolio

In [6]:
# Performance evaluator
def evaluate_strategy(individual, data):
    short_window = individual[0]
    long_window = individual[1]

    if short_window >= long_window:
        return -np.inf,

    signals = moving_average_crossover(data, short_window, long_window)
    portfolio = calculate_returns(data, signals)
    returns = portfolio['returns']
    sharpe_ratio = np.sqrt(252) * (returns.mean() / returns.std())
    return sharpe_ratio,

In [7]:
# Optimization function
def optimize_strategy(stock):
    # Fetch data
    data = fetch_data(stock, '2020-01-01', '2023-06-30')

    # Define the fitness and individual classes
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    # Initialize the toolbox
    toolbox = base.Toolbox()

    # We will search for optimal moving average windows between 1 and 100
    toolbox.register("attr_int", random.randint, 1, 100)

    # Our individuals will have two genes - one for short and one for long window
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=2)

    # The population will consist of 100 individuals
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Register the evaluation operator
    toolbox.register("evaluate", evaluate_strategy, data=data)

    # Register the crossover operator
    toolbox.register("mate", tools.cxTwoPoint)

    # Register a mutation operator with a probability to mutate of 0.05
    toolbox.register("mutate", tools.mutUniformInt, low=1, up=100, indpb=0.05)

    # Operator for selecting individuals for breeding the next generation
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Run the optimization
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof, verbose=True)

    print(f"Best individual for {stock} is: {hof[0]}\nwith fitness: {hof[0].fitness}")

    return hof[0], log

In [ ]:
# Dictionary to store the best strategy parameters for each stock
best_strategy_params = {}

# Loop through the list of stocks and run the optimization
for stock in fintech_stocks:
    best_strategy_params[stock] = optimize_strategy(stock)

In [10]:
from IPython.display import clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt

def plot_evolution(log):
    """
    Function to plot the evolution of populations.
    Args:
    log : DEAP Logbook
        The logbook object to plot.
    Returns:
    None
    """
    gen = log.select("gen")
    fit_mins = log.select("min")
    fit_avgs = log.select("avg")
    fit_maxs = log.select("max")

    fig, ax1 = plt.subplots()
    line1 = ax1.plot(gen, fit_mins, "b-", label="Minimum Fitness")
    ax1.set_xlabel("Generation")
    ax1.set_ylabel("Fitness", color="b")
    for tl in ax1.get_yticklabels():
        tl.set_color("b")

    ax2 = ax1.twinx()
    line2 = ax2.plot(gen, fit_avgs, "r-", label="Average Fitness")
    for tl in ax2.get_yticklabels():
        tl.set_color("r")

    lns = line1 + line2
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs, loc="center right")

    plt.show()

# Store the logbooks for each optimization in a dictionary
logbooks = {}
best_strategy_params = {}

for stock in fintech_stocks:
    best_strategy_params[stock], logbooks[stock] = optimize_strategy(stock)

def interactive_plot(stock):
    plot_evolution(logbooks[stock])

stocks_widget = widgets.Dropdown(
    options=fintech_stocks,
    description='Stock:',
)
interact(interactive_plot, stock=stocks_widget)


[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg 	min 	max     
0  	100   	-inf	-inf	0.722844
1  	55    	-inf	-inf	0.722844
2  	52    	-inf	-inf	0.801222
3  	64    	-inf	-inf	0.801222
4  	66    	-inf	-inf	0.801222
5  	61    	-inf	-inf	0.722844
6  	64    	0.636004	0.18174	0.77004 
7  	53    	-inf    	-inf   	0.77004 
8  	63    	-inf    	-inf   	0.77004 
9  	59    	-inf    	-inf   	0.77004 
10 	54    	-inf    	-inf   	0.77004 
11 	70    	-inf    	-inf   	0.77004 
12 	56    	-inf    	-inf   	0.806815
13 	57    	-inf    	-inf   	0.806815
14 	56    	-inf    	-inf   	0.806815
15 	54    	-inf    	-inf   	0.806815
16 	50    	-inf    	-inf   	0.806815
17 	56    	-inf    	-inf   	0.806815
18 	64    	0.806447	0.77004	0.806815
19 	58    	0.806815	0.806815	0.806815
20 	58    	-inf    	-inf    	0.806815
21 	61    	-inf    	-inf    	0.806815
22 	55    	-inf    	-inf    	0.806815
23 	54    	-inf    	-inf    	0.806815
24 	56    	-inf    	-inf    	0.806815
25 	54    	-inf    	-inf    	0.806815
26 	58    	-inf    	-inf    	0.806815
27 	6

/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg 	min 	max     
0  	100   	-inf	-inf	0.800052
1  	54    	-inf	-inf	0.97551 
2  	56    	-inf	-inf	0.977848
3  	63    	-inf	-inf	1.02648 
4  	47    	-inf	-inf	1.02648 
5  	67    	-inf	-inf	0.977848
6  	58    	0.881364	-0.0740787	0.977848
7  	69    	0.943969	-0.566848 	0.977848
8  	60    	-inf    	-inf      	0.977848
9  	52    	0.97736 	0.929069  	0.977848
10 	56    	0.961349	-0.0086624	0.977848
11 	57    	-inf    	-inf      	0.977848
12 	55    	-inf    	-inf      	0.977848
13 	58    	-inf    	-inf      	0.977848
14 	66    	-inf    	-inf      	0.977848
15 	54    	-inf    	-inf      	0.977848
16 	75    	-inf    	-inf      	0.977848
17 	57    	0.977848	0.977848  	0.977848
18 	64    	-inf    	-inf      	0.977848
19 	64    	-inf    	-inf      	0.977848
20 	66    	0.962299	-0.577064 	0.977848
21 	60    	0.973134	0.506459  	0.977848
22 	61    	0.973926	0.708003  	0.977848
23 	53    	0.970068	0.199871  	0.977848
24 	66    	-inf    	-inf      	0.977848
25 	69    	-inf    	-inf      

...

gen	nevals	avg 	min 	max    
0  	100   	-inf	-inf	1.14668
1  	56    	-inf	-inf	0.955158
2  	60    	-inf	-inf	1.14668 
3  	60    	-inf	-inf	1.20525 
4  	61    	-inf	-inf	1.2309  
5  	59    	-inf	-inf	1.2309  
6  	63    	-inf	-inf	1.2309  
7  	46    	1.15683	0.286917	1.2309  
8  	48    	1.18418	0.438896	1.2309  
9  	66    	-inf   	-inf    	1.2309  
10 	66    	1.20718	0.167823	1.2309  
11 	64    	1.2309 	1.2309  	1.2309  
12 	59    	-inf   	-inf    	1.2309  
13 	68    	-inf   	-inf    	1.2309  
14 	59    	1.2309 	1.2309  	1.2309  
15 	57    	-inf   	-inf    	1.2309  
16 	59    	-inf   	-inf    	1.2309  
17 	66    	-inf   	-inf    	1.2309  
18 	50    	1.22034	0.174236	1.2309  
19 	62    	-inf   	-inf    	1.2309  
20 	53    	1.21929	0.0692821	1.2309  
21 	54    	1.2309 	1.2309   	1.2309  
22 	66    	-inf   	-inf     	1.2309  
23 	54    	-inf   	-inf     	1.2309  
24 	59    	1.2309 	1.2309   	1.2309  
25 	65    	-inf   	-inf     	1.2309  
26 	54    	-inf   	-inf     	1.2309  
27 	70    	-inf

/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg 	min 	max    
0  	100   	-inf	-inf	1.37234
1  	62    	-inf	-inf	1.37234
2  	62    	-inf	-inf	1.38347
3  	64    	-inf	-inf	1.39291
4  	71    	-inf	-inf	1.39502
5  	50    	-inf	-inf	1.39502
6  	49    	1.282	0.0640141	1.39502
7  	61    	1.32656	0.735848 	1.39502
8  	64    	-inf   	-inf     	1.39502
9  	64    	-inf   	-inf     	1.39502
10 	59    	1.392  	1.238    	1.39502
11 	60    	-inf   	-inf     	1.39502
12 	54    	-inf   	-inf     	1.39502
13 	70    	1.39502	1.39502  	1.39502
14 	56    	1.38536	0.753817 	1.39502
15 	66    	-inf   	-inf     	1.39502
16 	59    	1.39357	1.24993  	1.39502
17 	70    	-inf   	-inf     	1.39502
18 	58    	1.38861	0.753817 	1.39502
19 	57    	-inf   	-inf     	1.42333
20 	66    	1.38004	0.596581 	1.42333
21 	61    	1.3911 	0.889872 	1.42333
22 	53    	1.3981 	1.33537  	1.42333
23 	49    	1.40578	1.39502  	1.42333
24 	70    	-inf   	-inf     	1.42333
25 	64    	1.42333	1.42333  	1.42333
26 	66    	-inf   	-inf     	1.42333
27 	66    	-inf   	-in

interactive(children=(Dropdown(description='Stock:', options=('V', 'MA', 'PYPL', 'SQ', 'ADP', 'GS', 'MS', 'JPM…

<function __main__.interactive_plot(stock)>